# Ensemble: Comb

This model is a combination of SES, Holts Linear Method and Holt's damped trend.  Comb was included to allow for a "standard" benchmark combination forecast for the more "complex" variants such as ARIMA+Prophet.

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')


#forecast error metrics
from forecast_tools.metrics import (mean_absolute_scaled_error, 
                                    root_mean_squared_error,
                                    symmetric_mean_absolute_percentage_error)

import statsmodels as sm
from statsmodels.tsa.statespace.exponential_smoothing import ExponentialSmoothing

import warnings
warnings.filterwarnings('ignore')

In [20]:
print(sm.__version__)

0.11.0


In [4]:
#ensemble learning
from amb_forecast.ensemble import (Ensemble, UnweightedVote)

# Data Input

The constants `TOP_LEVEL`, `STAGE`, `REGION`,`TRUST` and `METHOD` are used to control data selection and the directory for outputting results.  

> Output file is `f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv'.csv`.  where metric will be smape, rmse, mase, coverage_80 and coverage_95. Note: `REGION`: is also used to select the correct data from the input dataframe.

In [5]:
TOP_LEVEL = '../../../results/model_selection'
STAGE = 'stage1'
REGION = 'Trust'
METHOD = 'comb'

FILE_NAME = 'Daily_Responses_5_Years_2019_full.csv'

#split training and test data.
TEST_SPLIT_DATE = '2019-01-01'

#second subdivide: train and val
VAL_SPLIT_DATE = '2017-07-01'

#discard data after 2020 due to coronavirus
#this is the subject of a seperate study.
DISCARD_DATE = '2020-01-01'

In [6]:
#read in path
path = f'../../../data/{FILE_NAME}'

In [7]:
def pre_process_daily_data(path, index_col, by_col, 
                           values, dayfirst=False):
    '''
    Daily data is stored in long format.  Read in 
    and pivot to wide format so that there is a single 
    colmumn for each regions time series.
    '''
    df = pd.read_csv(path, index_col=index_col, parse_dates=True, 
                     dayfirst=dayfirst)
    df.columns = map(str.lower, df.columns)
    df.index.rename(str(df.index.name).lower(), inplace=True)
    
    clean_table = pd.pivot_table(df, values=values.lower(), 
                                 index=[index_col.lower()],
                                 columns=[by_col.lower()], aggfunc=np.sum)
    
    clean_table.index.freq = 'D'
    
    return clean_table

In [8]:
clean = pre_process_daily_data(path, 'Actual_dt', 'ORA', 'Actual_Value', 
                               dayfirst=False)
clean.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire
actual_dt,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0


## Train Test Split

In [9]:
def ts_train_test_split(data, split_date):
    '''
    Split time series into training and test data
    
    Parameters:
    -------
    data - pd.DataFrame - time series data.  Index expected as datatimeindex
    split_date - the date on which to split the time series
    
    Returns:
    --------
    tuple (len=2) 
    0. pandas.DataFrame - training dataset
    1. pandas.DataFrame - test dataset
    '''
    train = data.loc[data.index < split_date]
    test = data.loc[data.index >= split_date]
    return train, test

In [10]:
train, test = ts_train_test_split(clean, split_date=TEST_SPLIT_DATE)

#exclude data after 2020 due to coronavirus.
test, discard = ts_train_test_split(test, split_date=DISCARD_DATE)

#train split into train and validation
train, val = ts_train_test_split(train, split_date=VAL_SPLIT_DATE)

In [11]:
train.shape

(1279, 9)

In [12]:
val.shape

(549, 9)

#  Models in the Ensemble
1. SES
2. Holt's Linear Trend Method
3. Holt's Damped Trend

The class below is a 'wrapper' class that provides the same interfacew for all methods and works in the time series cross valiation code.  The prediction is simply the average of all methods.

In [26]:
class ExponentialSmoothingWrapper:
    '''
    Facade for statsmodels exponential smoothing models.  This wrapper
    provides a common interface for all models and allow interop with
    the custom time series cross validation code.
    '''
    def __init__(self, trend=False, damped_trend=False, seasonal=None):
        self._trend = trend
        self._seasonal= seasonal
        self._damped_trend = damped_trend

    def _get_resids(self):
        return self._fitted.resid

    def _get_preds(self):
        return self._fitted.fittedvalues

    def fit(self, train):
        '''
        Fit the model
        
        Parameters:
        train: array-like
            time series to fit.
        '''
        self._model = ExponentialSmoothing(endog=train,
                                          trend=self._trend, 
                                          damped_trend=self._damped_trend,
                                          seasonal=self._seasonal)
        self._fitted = self._model.fit()
        self._t = len(train)
    
    def predict(self, horizon, return_conf_int=False, alpha=0.2):
        '''
        Forecast the time series from the final point in the fitted series.
        
        Parameters:
        ----------
        
        horizon: int
            steps ahead to forecast 
            
        return_conf_int: bool, optional (default=False)
            Return prediction interval?  
            
        alpha: float
            Used if return_conf_int=True. 100(1-alpha) interval.
        '''
        
        forecast = self._fitted.get_forecast(horizon)
        
        mean_forecast = forecast.summary_frame()['mean'].to_numpy()
        
        if return_conf_int:
            df = forecast.summary_frame(alpha=alpha)
            pi = df[['mean_ci_lower', 'mean_ci_upper']].to_numpy()
            return mean_forecast, pi        
        else:
            return mean_forecast

    fittedvalues = property(_get_preds)
    resid = property(_get_resids)

# Example fitting and prediction with comb

In [27]:
#StateSpaceExponentialSmoothing is a class I have created to help with CV
#It is an adaptor class for the standard Exp Smoothing methods
model_1 = ExponentialSmoothingWrapper(trend=False, damped_trend=False)
model_2 = ExponentialSmoothingWrapper(trend=True)
model_3 = ExponentialSmoothingWrapper(trend=True, damped_trend=True)

In [28]:
estimators = {'ses': model_1, 'trend': model_2, 'damped_trend': model_3}
ens = Ensemble(estimators, UnweightedVote())

In [29]:
ens.fit(train[REGION])

In [30]:
H = 5
ens_preds = ens.predict(horizon=H)

In [31]:
ens_preds, pi = ens.predict(horizon=H, return_conf_int=True)

In [32]:
ens_preds

array([2150.02589006, 2150.05203479, 2150.07824001, 2150.10449364,
       2150.13078598])

In [33]:
pi

array([[2000.41163537, 2299.64014475],
       [1964.21206722, 2335.89200236],
       [1935.68854566, 2364.46793436],
       [1911.3518445 , 2388.85714278],
       [1889.75511668, 2410.50645528]])

## Time Series Cross Validation

`time_series_cv` implements rolling forecast origin cross validation for time series.  
It does not calculate forecast error, but instead returns the predictions, pred intervals and actuals in an array that can be passed to any forecast error function. (this is for efficiency and allows additional metrics to be calculated if needed).

In [36]:
def time_series_cv(model, train, val, horizons, alpha=0.2, step=1):
    '''
    Time series cross validation across multiple horizons for a single model.

    Incrementally adds additional training data to the model and tests
    across a provided list of forecast horizons. Note that function tests a
    model only against complete validation sets.  E.g. if horizon = 15 and 
    len(val) = 12 then no testing is done.  In the case of multiple horizons
    e.g. [7, 14, 28] then the function will use the maximum forecast horizon
    to calculate the number of iterations i.e if len(val) = 365 and step = 1
    then no. iterations = len(val) - max(horizon) = 365 - 28 = 337.
    
    Parameters:
    --------
    model - forecasting model

    error_func - function to measure forecast error

    train - np.array - vector of training data

    val - np.array - vector of validation data

    horizon - list of ints, forecast horizon e.g. [7, 14, 28] days

    step -- step taken in cross validation 
            e.g. 1 in next cross validation training data includes next point 
            from the validation set.
            e.g. 7 in the next cross validation training data includes next 7 points
            (default=1)
            
    Returns:
    -------
    np.array - vector of forecast errors from the CVs.
    '''
    cv_preds = [] #mean forecast
    cv_actuals = [] # actuals 
    cv_pis = [] #prediction intervals
    split = 0

    print('split => ', end="")
    for i in range(0, len(val) - max(horizons) + 1, step):
        split += 1
        print(f'{split}, ', end="")
                
        train_cv = np.concatenate([train, val[:i]], axis=0)
        model.fit(train_cv)
        
        #predict the maximum horizon 
        preds, pis = model.predict(horizon=len(val[i:i+max(horizons)]), 
                                   return_conf_int=True,
                                   alpha=alpha)
        
        cv_h_preds = []
        cv_test = []
        cv_h_pis = []
        
        for h in horizons:
            #store the h-step prediction
            cv_h_preds.append(preds[:h])
            #store the h-step actual value
            cv_test.append(val.iloc[i:i+h])    
            cv_h_pis.append(pis[:h])
                     
        cv_preds.append(cv_h_preds)
        cv_actuals.append(cv_test)
        cv_pis.append(cv_h_pis)
        
    print('done.\n')        
    return cv_preds, cv_actuals, cv_pis

## Custom functions for calculating CV scores for point predictions and coverage.

These functions have been written to work with the output of `time_series_cv`

In [37]:
def split_cv_error(cv_preds, cv_test, error_func):
    '''
    Forecast error in the current split
    
    Params:
    -----
    cv_preds, np.array
        Split predictions
        
    
    cv_test: np.array
        acutal ground truth observations
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
        np.ndarray
            cross validation errors for split
    '''
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = error_func(cv_test[split], cv_preds[split])
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv(cv_preds, cv_test, error_func):
    '''
    Forecast errors by forecast horizon
    
    Params:
    ------
    cv_preds: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    cv_test: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
    np.ndarray
        
    '''
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error(cv_preds[h], cv_test[h], error_func)
        horizon_errors.append(split_errors)

    return np.array(horizon_errors)

def split_coverage(cv_test, cv_intervals):
    n_splits = len(cv_test)
    cv_errors = []
        
    for split in range(n_splits):
        val = np.asarray(cv_test[split])
        lower = cv_intervals[split].T[0]
        upper = cv_intervals[split].T[1]
        
        coverage = len(np.where((val > lower) & (val < upper))[0])
        coverage = coverage / len(val)
        
        cv_errors.append(coverage)
        
    return np.array(cv_errors)
    
    
def prediction_int_coverage_cv(cv_test, cv_intervals):
    cv_test = np.array(cv_test)
    cv_intervals = np.array(cv_intervals)
    n_horizons = len(cv_test)    
    
    horizon_coverage = []
    for h in range(n_horizons):
        split_coverages = split_coverage(cv_test[h], cv_intervals[h])
        horizon_coverage.append(split_coverages)

    return np.array(horizon_coverage)  

In [38]:
def split_cv_error_scaled(cv_preds, cv_test, y_train):
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = mean_absolute_scaled_error(cv_test[split], cv_preds[split], 
                                                y_train, period=7)
        
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv_scaled(cv_preds, cv_test, y_train):
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error_scaled(cv_preds[h], cv_test[h], y_train)
        horizon_errors.append(split_errors)
        
    return np.array(horizon_errors)

### Get model and conduct tscv.

In [39]:
def get_comb():
    '''
    Create ensemble model
    '''
    model_1 = StateSpaceExponentialSmoothing(trend=False, damped_trend=False)
    model_2 = StateSpaceExponentialSmoothing(trend=True)
    model_3 = StateSpaceExponentialSmoothing(trend=True, damped_trend=True)
    estimators = {'ses': model_1, 'trend': model_2, 'damped_trend': model_3}
    return Ensemble(estimators, UnweightedVote())
    

In [40]:
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
model = get_comb()

results = time_series_cv(model, train[REGION], val[REGION], horizons, 
                         alpha=0.2, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [41]:
cv_preds, cv_test, cv_intervals = results
#CV point predictions smape
cv_errors = forecast_errors_cv(cv_preds, cv_test, 
                               symmetric_mean_absolute_percentage_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,4.144404,4.387557,4.482939,4.567904,4.711091,4.831040,4.914774,4.998533,5.123216,5.208575,5.291513,5.387491,6.123737
std,1.492429,1.432373,1.333328,1.163322,1.205066,1.235896,1.181915,1.142638,1.138796,1.084850,1.005869,1.047530,1.747465
min,1.783327,2.400819,2.533738,2.849314,3.110957,3.380114,3.400292,3.627686,3.705861,3.765601,3.866121,3.989448,4.344736
25%,3.082884,3.624873,3.730435,3.952215,3.890757,4.049973,4.096396,4.110047,4.161016,4.296811,4.582256,4.627237,4.552741
50%,4.216862,4.159924,4.333012,4.352103,4.456228,4.480138,4.756983,4.797748,4.950297,5.105147,5.271748,5.348539,5.610691
75%,4.818735,4.719601,4.857971,4.662025,5.311139,5.358166,5.429152,5.377739,5.714238,5.970226,5.948374,5.940844,7.050673
max,8.936688,8.949729,8.980781,8.500660,8.773975,8.732895,8.647544,8.394020,8.672909,8.428173,8.119109,8.436276,10.739689


In [42]:
#output sMAPE results to file
metric = 'smape'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-comb_smape.csv


In [43]:
#CV point predictions rmse
cv_errors = forecast_errors_cv(cv_preds, cv_test, root_mean_squared_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,116.026222,124.735794,130.024436,133.744223,138.366575,142.342950,145.735258,148.688279,152.245203,154.950908,157.568809,160.322071,173.027134
std,42.166203,42.198608,39.495637,36.559464,37.218912,37.958186,36.034207,34.945287,33.765041,31.578792,28.025813,27.079460,39.831774
min,46.618512,72.843238,76.824240,91.125853,94.781962,96.130241,97.633007,106.669956,108.718770,104.615344,107.175477,111.059828,128.098587
25%,85.289554,100.615785,106.938011,110.388183,108.328504,115.894182,114.093023,116.842668,119.950491,134.037732,141.630531,145.646319,137.413187
50%,112.886163,116.403959,120.862203,121.641565,129.204218,133.055584,136.099152,141.010069,153.949997,154.526077,154.339609,156.354794,166.053986
75%,139.239252,137.553280,145.318354,143.450635,151.259772,163.334095,169.530709,165.021614,167.329742,169.726308,176.439025,182.600954,193.672368
max,259.088004,250.807024,238.009558,224.213457,227.454278,245.971990,230.194700,228.163770,225.293410,220.559616,214.135401,220.543392,275.251753


In [44]:
#output rmse
metric = 'rmse'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-comb_rmse.csv


In [45]:
#mase
cv_errors = forecast_errors_cv_scaled(cv_preds, cv_test, train[REGION])
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,1.122101,1.188923,1.216383,1.240823,1.280217,1.313480,1.336937,1.360145,1.394237,1.417573,1.440587,1.467164,1.652850
std,0.450470,0.438261,0.410775,0.365715,0.371286,0.376176,0.356261,0.341648,0.335756,0.315250,0.287835,0.294382,0.469694
min,0.477044,0.654026,0.705157,0.799043,0.877631,0.917456,0.926158,0.979531,1.002695,1.011391,1.009348,1.034784,1.182472
25%,0.845205,0.959518,0.982888,1.019405,1.016767,1.096668,1.052117,1.072047,1.119535,1.193911,1.234461,1.252797,1.236548
50%,1.074557,1.109406,1.127614,1.136101,1.151357,1.171257,1.277179,1.317182,1.323615,1.341009,1.452884,1.469867,1.530274
75%,1.296387,1.267473,1.291686,1.256845,1.411692,1.500616,1.515765,1.519339,1.546098,1.631714,1.628111,1.619943,1.910424
max,2.728739,2.627448,2.601284,2.454834,2.519974,2.502644,2.472801,2.399179,2.471969,2.403369,2.316685,2.402194,3.019100


In [46]:
#output rmse
metric = 'mase'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-comb_mase.csv


In [47]:
#80% PIs
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.841270,0.910053,0.936508,0.947090,0.954497,0.961199,0.965986,0.967593,0.970018,0.973016,0.974988,0.977072,0.994622
std,0.139146,0.096225,0.064702,0.054588,0.047717,0.039703,0.033961,0.031338,0.027959,0.025163,0.022473,0.020600,0.004835
min,0.571429,0.571429,0.714286,0.785714,0.828571,0.857143,0.877551,0.892857,0.904762,0.914286,0.922078,0.928571,0.983562
25%,0.714286,0.857143,0.904762,0.928571,0.942857,0.952381,0.959184,0.955357,0.952381,0.957143,0.961039,0.964286,0.991781
50%,0.857143,0.928571,0.952381,0.964286,0.971429,0.976190,0.979592,0.982143,0.968254,0.971429,0.974026,0.976190,0.994521
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.991071,0.992063,0.992857,0.987013,0.988095,0.997260
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [48]:
#output 80% PI coverage
metric = 'coverage_80'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-comb_coverage_80.csv


### Rerun for 95% PI coverage

In [49]:
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
model = get_comb()

results = time_series_cv(model, train[REGION], val[REGION], horizons, 
                         alpha=0.05, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [50]:
#95% PIs
cv_preds, cv_test, cv_intervals = results
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.957672,0.973545,0.982363,0.985450,0.987302,0.989418,0.990930,0.991402,0.992357,0.993122,0.993747,0.994268,0.998681
std,0.077387,0.052966,0.035311,0.026688,0.021459,0.017883,0.015328,0.014329,0.012737,0.011463,0.010421,0.009553,0.002198
min,0.714286,0.785714,0.857143,0.892857,0.914286,0.928571,0.938776,0.946429,0.952381,0.957143,0.961039,0.964286,0.991781
25%,0.928571,0.964286,0.976190,0.964286,0.971429,0.976190,0.979592,0.982143,0.984127,0.985714,0.987013,0.988095,0.997260
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [51]:
#output 95% PI coverage
metric = 'coverage_95'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-comb_coverage_95.csv


# End